# [STARTER] Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [2]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [32]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv
from pydantic import BaseModel,Field
from typing import Annotated,List,Dict
import pandas as pd

from rich.console import Console
from rich.markdown import Markdown
console=Console()

In [ ]:
# TODO: Create a .env file with the following variables
OPENAI_API_KEY="your-openai-api-key"
TAVILY_API_KEY="your-tavily-api-key"
CHROMA_OPEN_API_KEY="your-chroma-api-key"

In [34]:
# TODO: Load environment variables
load_dotenv()

True

### VectorDB Instance

In [35]:
# TODO: Instantiate your ChromaDB Client
# Choose any path you want
chroma_client = chromadb.PersistentClient(path="chromadb")

### Collection

In [36]:
# TODO: Pick one embedding function
# If picking something different than openai, 
# make sure you use the same when loading it
#embedding_fn = embedding_functions.OpenAIEmbeddingFunction()
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    )

In [37]:
# TODO: Create a collection
# Choose any name you want
# Create a collection
if "udaplay" not in [c.name for c in chroma_client.list_collections()]:
    collection = chroma_client.create_collection(
        name="udaplay",
        embedding_function=embedding_fn
        )
else:
    collection = chroma_client.get_collection("udaplay")

### Add documents

In [38]:
# Make sure you have a directory "project/starter/games"
data_dir = "games"

for file_name in sorted(os.listdir(data_dir)):
    if not file_name.endswith(".json"):
        continue

    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]

    collection.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )

In [39]:
# Access each list item as model objects, ie result.Name, result.Description, ect...
all_records = collection.get()
print(all_records.keys())

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas'])


In [40]:
class retrievedDocument(BaseModel):
    """Pydantic data model for retrieved documents from the vectorstore"""
    Platform: Annotated[str, Field(description="Game platform (PC, Xbox, Wii...)")]
    Name: Annotated[str, Field(description="Title of the game")]
    YearOfRelease: Annotated[int, Field(description="Year of release")]
    Description: Annotated[str, Field(description="One-liner game description")]

In [41]:
metadatas = all_records['metadatas']
platforms=[];names=[];years=[];descriptions=[]
# Use pydantic data model
# Now you can access them as model objects, ie result.Name, result.Description, ect...
documents = [retrievedDocument(**metadata) for metadata in metadatas]

In [42]:
console.print(f"{len(documents)} retrieved from the database. Examples: ", style='bold yellow')
documents[:3]

48 retrieved from the database. Examples: 

[retrievedDocument(Platform='PlayStation 1', Name='Gran Turismo', YearOfRelease=1997, Description='A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.'),
 retrievedDocument(Platform='PlayStation 2', Name='Grand Theft Auto: San Andreas', YearOfRelease=2004, Description="An expansive open-world game set in the fictional state of San Andreas, following the story of Carl 'CJ' Johnson."),
 retrievedDocument(Platform='PlayStation 3', Name='Gran Turismo 5', YearOfRelease=2010, Description='A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.')]

In [43]:
def print_record(collectionName:str, id:int):
    """
    retrieves a item from a chromadb collection
    inputs:
        - collection name (str)
        - id (int)
    outputs:
        - record id (dict) 
    """

    chroma_client = chromadb.PersistentClient(path="chromadb")

    if collectionName in [c.name for c in chroma_client.list_collections()]:
        collection = chroma_client.get_collection(collectionName)
    else:
        console.print(f"No collection '{collectionName}' found", style='bold red')
        return None
    
    all_records = collection.get()
    metadatas = all_records['metadatas']

    if f'{id:03}' in all_records['ids']:
        return metadatas[id]
    else:
        console.print(f"Item '{id}' does not exist", style='bold red')
        return None

In [44]:
print_record('udaplay', 1)

{'Genre': 'Action-adventure',
 'Publisher': 'Rockstar Games',
 'Platform': 'PlayStation 2',
 'Description': "An expansive open-world game set in the fictional state of San Andreas, following the story of Carl 'CJ' Johnson.",
 'YearOfRelease': 2004,
 'Name': 'Grand Theft Auto: San Andreas'}

In [45]:
def retrieve_all_games()->List[retrievedDocument]:
    """
    Retrieves all records from the database
    Output:
    You'll receive all records as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """
    chroma_client = chromadb.PersistentClient(path="chromadb")
    collection = chroma_client.get_collection("udaplay")
    # Retrieve all records
    all_records = collection.get()

    if all_records:
        metadatas = all_records['metadatas']
        platforms=[];names=[];years=[];descriptions=[]
        # Use pydantic data model
        # Now you can access them as model objects, ie result.Name, result.Description, ect...
        documents = [retrievedDocument(**metadata) for metadata in metadatas]
        
    return documents

In [46]:
temp={}
all_records = retrieve_all_games()
for i in range(len(all_records)):
    temp[i] = pd.DataFrame(json.loads(all_records[i].model_dump_json()), index=[i])
records_df = pd.concat(temp,ignore_index=True)
records_df

,Platform,Name,YearOfRelease,Description
0,PlayStation 1,Gran Turismo,1997,A realistic racing simulator featuring a wide ...
1,PlayStation 2,Grand Theft Auto: San Andreas,2004,An expansive open-world game set in the fictio...
2,PlayStation 3,Gran Turismo 5,2010,A comprehensive racing simulator featuring a v...
3,PlayStation 4,Marvel's Spider-Man,2018,An open-world superhero game that lets players...
4,PlayStation 5,Marvel's Spider-Man 2,2023,"The sequel to the acclaimed Spider-Man game, f..."
5,Game Boy Color,Pokémon Gold and Silver,1999,Second-generation Pokémon games introducing ne...
6,Game Boy Advance,Pokémon Ruby and Sapphire,2002,Third-generation Pokémon games set in the Hoen...
7,Super Nintendo Entertainment System (SNES),Super Mario World,1990,A classic platformer where Mario embarks on a ...
8,Nintendo 64,Super Mario 64,1996,A groundbreaking 3D platformer that set new st...
9,GameCube,Super Smash Bros. Melee,2001,A crossover fighting game featuring characters...
